In [ ]:
import pandas as pd
from calendar import monthrange

dataframe_list = []

for month in range(1, 13):
    year = 2019

    monthly_data = pd.read_parquet(f'{year}/{year}-{month:02d}.parquet')

    monthly_data = monthly_data[
        ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID',
         'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'tip_amount', 'total_amount']
    ]

    filter_columns = ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'tip_amount', 'total_amount']
    percentile = .01

    filter_string = ' & '.join(
        (
            *(f'({column}.quantile({percentile}) <= {column}) & ({column} <= {column}.quantile({1 - percentile}))' for column in filter_columns),
            f'("{year}, {month}, 1" <= tpep_pickup_datetime) & (tpep_pickup_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
            f'("{year}, {month}, 1" <= tpep_dropoff_datetime) & (tpep_dropoff_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
            'tpep_pickup_datetime <= tpep_dropoff_datetime'
        )
    )

    monthly_data = monthly_data.query(filter_string)
    
    dataframe_list.append(monthly_data)

yearly_data = pd.concat(dataframe_list)

yearly_data.to_parquet(f'{year}/{year}.parquet')